<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#!/usr/bin/env python3
"""
Colab-ready launcher for MY-AI-Gizmo (fixed)
- Downloads repo if missing
- Persists data to Drive
- Runs installer (start_linux.sh) with live streaming and heartbeat
- Launches server via wrapper that forces MPL backend to Agg
- Captures Gradio public URL and writes it to Drive_root/public_url.txt
"""

import os
import subprocess
import shutil
import re
import time
import threading
from pathlib import Path

# ---------- Config ----------
REPO_ZIP = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")
LOG_DIR = DRIVE_ROOT / "logs"
MPL_CONFIG_DIR = DRIVE_ROOT / "matplotlib"
HEARTBEAT_INTERVAL = 30
# Optional: set MODEL_TO_DOWNLOAD environment variable or change here:
MODEL_TO_DOWNLOAD = os.environ.get("MODEL_TO_DOWNLOAD", None)
# ----------------------------

# Utilities
def run(cmd, cwd=None, env=None, check=False):
    return subprocess.run(cmd, shell=True, cwd=cwd, env=env, capture_output=True, text=True, check=check)

def stream_and_capture(cmd, cwd=None, env=None, log_path=None, url_output_file=None):
    """
    Stream stdout/stderr from cmd, append to log_path.
    Detect and return first URL seen (prefer gradio.* domains).
    """
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                            cwd=cwd, env=env, text=True, bufsize=1)

    last_output = time.time()
    stop_flag = threading.Event()
    captured_url = None

    url_patterns = [
        re.compile(r"(https?://[^\s)'\"]+gradio\.\w+[^\s)'\"]*)", re.IGNORECASE),
        re.compile(r"(https?://[^\s)'\"]+\.gradio\.live[^\s)'\"]*)", re.IGNORECASE),
        re.compile(r"(https?://[^\s)'\"]+:\d+)", re.IGNORECASE),
        re.compile(r"(https?://[^\s)'\"]+)", re.IGNORECASE),
    ]

    def heartbeat():
        while not stop_flag.wait(HEARTBEAT_INTERVAL):
            if time.time() - last_output >= HEARTBEAT_INTERVAL:
                msg = f"[heartbeat] still working... (no output for ~{HEARTBEAT_INTERVAL}s)\n"
                print(msg, end='')
                if log_path:
                    try:
                        with open(log_path, "a", encoding="utf-8") as f:
                            f.write(msg)
                    except Exception:
                        pass

    hb = threading.Thread(target=heartbeat, daemon=True)
    hb.start()

    logfile = None
    try:
        if log_path:
            logfile = open(log_path, "a", encoding="utf-8")
        for line in proc.stdout:
            last_output = time.time()
            print(line, end='')
            if logfile:
                logfile.write(line)
            if not captured_url:
                for pat in url_patterns:
                    m = pat.search(line)
                    if m:
                        captured_url = m.group(1)
                        # prefer gradio hosts; if found, break immediately
                        if 'gradio' in captured_url.lower():
                            break
                # If captured_url contains trailing punctuation, strip it
                if captured_url:
                    captured_url = captured_url.rstrip(').,\'"')
                    # write URL to file if requested
                    if url_output_file:
                        try:
                            Path(url_output_file).write_text(captured_url)
                        except Exception:
                            pass
    except Exception as e:
        print(f"[error] streaming error: {e}")
    finally:
        proc.wait()
        stop_flag.set()
        hb.join(timeout=1)
        if logfile:
            logfile.close()
    return proc.returncode, captured_url

# Ensure Drive is mounted (Colab)
IN_COLAB = False
try:
    import google.colab
    from google.colab import drive as gdrive
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    try:
        gdrive.mount("/content/drive", force_remount=False)
    except Exception:
        pass

# Prepare Drive folders
for p in (DRIVE_ROOT, LOG_DIR, MPL_CONFIG_DIR):
    try:
        p.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass

# Ensure repo is present (download if missing)
def download_repo_if_missing():
    if WORK_DIR.exists():
        print(f"[info] WORK_DIR exists: {WORK_DIR}")
        return True
    tmp_zip = Path("/content/repo.zip")
    try:
        tmp_zip.unlink()
    except Exception:
        pass
    print("[info] downloading repo zip...")
    ok = False
    for cmd in (f"wget -q -O {tmp_zip} {REPO_ZIP}", f"curl -s -L -o {tmp_zip} {REPO_ZIP}"):
        run(cmd)
        if tmp_zip.exists() and tmp_zip.stat().st_size > 1000:
            ok = True
            break
    if not ok:
        print("[error] download failed")
        return False
    print("[info] extracting...")
    run(f"unzip -q {tmp_zip} -d /content")
    found = next(Path("/content").glob("MY-AI-Gizmo-working-*"), None)
    if not found:
        print("[error] extracted but expected folder not found")
        return False
    found.rename(WORK_DIR)
    print(f"[info] repo extracted to {WORK_DIR}")
    return True

if not download_repo_if_missing() and not WORK_DIR.exists():
    raise SystemExit("Failed to obtain repo")

# Switch to repo
os.chdir(str(WORK_DIR))

# Create symlinks to persist data in Drive
links_map = [
    ("models", "models"),
    ("loras", "loras"),
    ("user_data/characters", "characters"),
    ("user_data/presets", "presets"),
    ("user_data/settings.yaml", "settings/settings.yaml"),
    ("user_data/settings.json", "settings/settings.json"),
    ("user_data/chat", "chat-history"),
    ("outputs", "outputs"),
]
def _remove_path_safe(p: Path):
    try:
        if p.is_symlink():
            p.unlink()
        elif p.is_dir():
            shutil.rmtree(p)
        elif p.exists():
            p.unlink()
    except Exception:
        pass

for local, drive_folder in links_map:
    drive_path = DRIVE_ROOT / drive_folder
    try:
        if drive_path.suffix:  # file
            drive_path.parent.mkdir(parents=True, exist_ok=True)
            if not drive_path.exists():
                # if it's a settings file, create an empty placeholder
                drive_path.write_text("", encoding="utf-8")
        else:
            drive_path.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass

    local_path = WORK_DIR / local
    try:
        _remove_path_safe(local_path)
    except Exception:
        pass
    # Try symlink, fallback to copy
    try:
        local_path.parent.mkdir(parents=True, exist_ok=True)
        os.symlink(str(drive_path), str(local_path), target_is_directory=drive_path.is_dir())
    except Exception:
        try:
            if drive_path.is_dir():
                shutil.copytree(drive_path, local_path)
            else:
                local_path.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(drive_path, local_path)
        except Exception:
            pass

# Ensure settings file exists and copy to repo user_data if Drive settings present
drive_settings = DRIVE_ROOT / "settings" / "settings.yaml"
local_settings = WORK_DIR / "user_data" / "settings.yaml"
local_settings.parent.mkdir(parents=True, exist_ok=True)
try:
    if drive_settings.exists() and drive_settings.stat().st_size > 0:
        shutil.copy2(drive_settings, local_settings)
    else:
        if not local_settings.exists() or local_settings.stat().st_size == 0:
            local_settings.write_text("# minimal\nlisten: true\nshare: true\n", encoding="utf-8")
            drive_settings.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(local_settings, drive_settings)
except Exception:
    pass

# Installer: run start_linux.sh if env missing
env_marker = WORK_DIR / "installer_files" / "env" / "bin" / "python"
start_sh = WORK_DIR / "start_linux.sh"
installer_log = LOG_DIR / f"installer_{int(time.time())}.log"
# Prepare env flags (tries to ensure fast but complete install)
install_env = os.environ.copy()
install_env["MPLBACKEND"] = "Agg"
install_env["MPLCONFIGDIR"] = str(MPL_CONFIG_DIR)
install_env.update({
    "GPU_CHOICE": "A",
    "LAUNCH_AFTER_INSTALL": "FALSE",   # install only
    "INSTALL_EXTENSIONS": "FALSE",
    "SKIP_LLAMACPP_BUILD": "TRUE",     # skip llama.cpp C compile to avoid long delays
    "SKIP_TORCH_TEST": "TRUE",
    "FORCE_CUDA": "FALSE",
})

if env_marker.exists():
    print("[info] virtualenv exists; skipping installer")
else:
    if start_sh.exists():
        print("[info] running installer (this may take several minutes). installer log:", installer_log)
        rc, _ = None, None
        try:
            rc, _ = stream_and_capture("bash start_linux.sh", cwd=str(WORK_DIR), env=install_env, log_path=str(installer_log))
        except TypeError:
            # older Python signature fallback
            rc = stream_and_capture("bash start_linux.sh", cwd=str(WORK_DIR), env=install_env, log_path=str(installer_log))
        if rc != 0:
            print(f"[warn] installer exit code {rc}. Check log: {installer_log}")
    else:
        print("[warn] start_linux.sh not found; cannot run installer automatically. You may need to run dependency installation manually.")

# Optional: run repo's download-model.py if provided and requested
if MODEL_TO_DOWNLOAD:
    dl_log = LOG_DIR / f"download_model_{int(time.time())}.log"
    download_script = WORK_DIR / "download-model.py"
    if download_script.exists():
        print(f"[info] running download-model.py for {MODEL_TO_DOWNLOAD}; log: {dl_log}")
        rc, _ = stream_and_capture(f'python "{download_script}" "{MODEL_TO_DOWNLOAD}"', cwd=str(WORK_DIR), env=os.environ.copy(), log_path=str(dl_log))
        if rc != 0:
            print(f"[warn] download-model.py exit code {rc}; see {dl_log}")
    else:
        print("[warn] download-model.py not found; skipping model download")

# Prepare server wrapper that forces MPL backend inside the server process
wrapper_path = WORK_DIR / "_run_server_with_agg.py"
wrapper_code = f"""# wrapper - auto-generated to force MPL backend to Agg
import os, runpy
os.environ['MPLBACKEND'] = 'Agg'
os.environ['MPLCONFIGDIR'] = r'{MPL_CONFIG_DIR}'
try:
    import matplotlib
    matplotlib.use('Agg', force=True)
except Exception:
    pass
runpy.run_path('server.py', run_name='__main__')
"""
try:
    wrapper_path.write_text(wrapper_code, encoding="utf-8")
    wrapper_path.chmod(0o755)
except Exception:
    pass

# Launch the server via the wrapper; capture and persist public URL
server_log = LOG_DIR / f"server_{int(time.time())}.log"
public_url_file = DRIVE_ROOT / "public_url.txt"

python_exe = str(env_marker) if env_marker.exists() else "python3"
launch_cmd = f'{python_exe} -u "{wrapper_path.name}"'
print("[info] launching server; log:", server_log)
rc, captured_url = None, None
try:
    rc, captured_url = stream_and_capture(launch_cmd, cwd=str(WORK_DIR), env=dict(os.environ, MPLBACKEND="Agg", MPLCONFIGDIR=str(MPL_CONFIG_DIR)), log_path=str(server_log), url_output_file=str(public_url_file))
except TypeError:
    # older signature fallback
    rc = stream_and_capture(launch_cmd, cwd=str(WORK_DIR), env=dict(os.environ, MPLBACKEND="Agg", MPLCONFIGDIR=str(MPL_CONFIG_DIR)), log_path=str(server_log), url_output_file=str(public_url_file))

if captured_url:
    try:
        print("\n" + "="*60)
        print("[success] Gradio URL captured:", captured_url)
        print("[info] saved to:", public_url_file)
        public_url_file.write_text(captured_url)
    except Exception:
        pass
else:
    print("[warn] no URL captured from server output. Check server log:", server_log)

if rc != 0:
    print(f"[warn] server exited with code {rc}. See log: {server_log}")
else:
    print("[info] server terminated normally. See log:", server_log)

print("\n[done] Persistent data and logs at:", DRIVE_ROOT)


✅ RECOMMENDED MODELS (COPY EXACTLY)
🔹 BEST GENERAL CHAT (START HERE)

Llama-2-7B-Chat

Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf

🔹 FAST + LIGHT (LOW RAM)

TinyLlama-1.1B-Chat

Repo: TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF
File: tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

🔹 STRONG CHAT (BETTER THAN LLAMA-2)

Mistral-7B-Instruct

Repo: TheBloke/Mistral-7B-Instruct-v0.2-GGUF
File: mistral-7b-instruct-v0.2.Q4_K_M.gguf

🔹 CODING MODEL

Code LLaMA-7B

Repo: TheBloke/CodeLlama-7B-GGUF
File: codellama-7b.Q4_K_M.gguf

🔹 ROLEPLAY / STORY

MythoMax-L2-13B (needs more RAM)

Repo: TheBloke/MythoMax-L2-13B-GGUF
File: mythomax-l2-13b.Q4_K_M.gguf

🔹 VERY FAST / TEST MODEL

Phi-2 (2.7B)

Repo: TheBloke/phi-2-GGUF
File: phi-2.Q4_K_M.gguf

⚙️ WHAT LOADER TO USE (IMPORTANT)

For ALL models above:

Loader: llama.cpp


Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf
